# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,13.74,68,100,2.84,AU,1691723043
1,1,edinburgh of the seven seas,-37.0676,-12.3116,12.14,70,97,12.43,SH,1691723043
2,2,afaahiti,-17.7500,-149.2833,25.59,69,100,8.53,PF,1691723043
3,3,port-aux-francais,-49.3500,70.2167,3.28,77,44,16.48,TF,1691723044
4,4,utrik,11.2278,169.8474,28.29,76,10,6.25,MH,1691723044


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] <= 26.66) &
                                 (city_data_df["Max Temp"] >= 21.11) &
                                 (city_data_df["Wind Speed"] <= 3.5) &
                                 (city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,sirte,31.2089,16.5887,24.83,68,0,3.40,LY,1691723049
37,37,'ain el hadjar,34.7585,0.1453,22.63,42,0,0.74,DZ,1691723052
48,48,hamilton,39.1834,-84.5333,22.69,86,0,2.57,US,1691723055
140,140,sugar hill,34.1065,-84.0335,25.27,89,0,0.00,US,1691722923
179,179,teodoro sampaio,-22.5325,-52.1675,23.04,35,0,2.84,BR,1691723001
181,181,ghat,24.9647,10.1728,26.66,24,0,1.26,LY,1691723087
224,224,motilla del palancar,39.5709,-1.9068,22.00,47,0,2.50,ES,1691722662
259,259,vila velha,-20.3297,-40.2925,21.97,83,0,2.06,BR,1691722826
291,291,glendive,47.1053,-104.7125,24.54,56,0,3.09,US,1691723125
325,325,illizi,26.4833,8.4667,25.89,18,0,1.35,DZ,1691722941


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Source 1 used here
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,sirte,LY,31.2089,16.5887,68,
37,'ain el hadjar,DZ,34.7585,0.1453,42,
48,hamilton,US,39.1834,-84.5333,86,
140,sugar hill,US,34.1065,-84.0335,89,
179,teodoro sampaio,BR,-22.5325,-52.1675,35,
181,ghat,LY,24.9647,10.1728,24,
224,motilla del palancar,ES,39.5709,-1.9068,47,
259,vila velha,BR,-20.3297,-40.2925,83,
291,glendive,US,47.1053,-104.7125,56,
325,illizi,DZ,26.4833,8.4667,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = hotel_df.loc[index, "Lat"] 
    longitude = hotel_df.loc[index, "Lng"] 
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sirte - nearest hotel: فندق المدينة-سرت
'ain el hadjar - nearest hotel: Hôtel Forsane
hamilton - nearest hotel: North Vista Manor
sugar hill - nearest hotel: Holiday Inn Express & Suites
teodoro sampaio - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
motilla del palancar - nearest hotel: Hotel del Sol
vila velha - nearest hotel: Hotel Prainha
glendive - nearest hotel: La Quinta Inn & Suites by Wyndham Glendive
illizi - nearest hotel: بوناقة
abu al matamir - nearest hotel: No hotel found
bovalino - nearest hotel: No hotel found
sterling - nearest hotel: Residence Inn by Marriott Dulles Airport at Dulles 28 Centre
enoch - nearest hotel: Travelodge by Wyndham Cedar City
troy - nearest hotel: Embassy Suites Hotel
caxias - nearest hotel: Hjem
camapua - nearest hotel: No hotel found
khovd - nearest hotel: Khovd hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
25,sirte,LY,31.2089,16.5887,68,فندق المدينة-سرت
37,'ain el hadjar,DZ,34.7585,0.1453,42,Hôtel Forsane
48,hamilton,US,39.1834,-84.5333,86,North Vista Manor
140,sugar hill,US,34.1065,-84.0335,89,Holiday Inn Express & Suites
179,teodoro sampaio,BR,-22.5325,-52.1675,35,No hotel found
181,ghat,LY,24.9647,10.1728,24,فندق تاسيلي
224,motilla del palancar,ES,39.5709,-1.9068,47,Hotel del Sol
259,vila velha,BR,-20.3297,-40.2925,83,Hotel Prainha
291,glendive,US,47.1053,-104.7125,56,La Quinta Inn & Suites by Wyndham Glendive
325,illizi,DZ,26.4833,8.4667,18,بوناقة


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.0,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)